In [ ]:
import geppy as gep
import operator

pset = gep.PrimitiveSet('main', input_names=['x', 'y'])
pset.add_function(max, 2)
pset.add_function(operator.add, 2)
pset.add_function(operator.mul, 2)
pset.add_constant_terminal(3)

In [ ]:
from deap import creator, base

creator.create("FitnessMax", base.Fitness, weights=(1,))
creator.create('Individual', gep.Chromosome, fitness=creator.FitnessMax)

In [ ]:
from deap import tools

h = 7   # head length
n_genes = 2
toolbox = gep.Toolbox()

toolbox.register('gene_gen', gep.Gene, pset=pset, head_length=h)
toolbox.register('individual', creator.Individual, gene_gen=toolbox.gene_gen, n_genes=n_genes, linker=operator.add)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [ ]:
toolbox.register('compile', gep.compile_, pset=pset)

In [ ]:
def evaluate(individual):
        func = toolbox.compile(individual)
        # inserting x and y into func and
        # compute the fitness of this individual
        # ....
        fitness = 1
        return fitness,
toolbox.register('evaluate', evaluate)

In [ ]:
toolbox.register('select', tools.selRoulette)

## general mutations whose aliases start with 'mut'
# We can specify the probability for an operator with the .pbs property
toolbox.register('mut_uniform', gep.mutate_uniform, pset=pset, ind_pb=2 / (2 * h + 1))
toolbox.pbs['mut_uniform'] = 1
# Alternatively, assign the probability along with registration using the pb keyword argument
toolbox.register('mut_invert', gep.invert, pb=0.1)
toolbox.register('mut_is_ts', gep.is_transpose, pb=0.1)
toolbox.register('mut_ris_ts', gep.ris_transpose, pb=0.1)
toolbox.register('mut_gene_ts', gep.gene_transpose, pb=0.1)

## general crossover whose aliases start with 'cx'
toolbox.register('cx_1p', gep.crossover_one_point, pb=0.1)
toolbox.pbs['cx_1p'] = 0.4   # just show that the probability can be overwritten
toolbox.register('cx_2p', gep.crossover_two_point, pb=0.2)
toolbox.register('cx_gene', gep.crossover_gene, pb=0.1)


In [ ]:
import numpy
stats = tools.Statistics(key=lambda ind: ind.fitness.values[0])
stats.register("avg", numpy.mean)
stats.register("std", numpy.std)
stats.register("min", numpy.min)
stats.register("max", numpy.max)

In [ ]:
hof = tools.HallOfFame(3)
# size of population and number of generations
n_pop = 100
n_gen = 100

pop = toolbox.population(n=n_pop)

# start evolution
pop, log = gep.gep_simple(pop, toolbox, n_generations=n_gen, n_elites=1,
        stats=stats, hall_of_fame=hof, verbose=True)

In [ ]:
best_individual = hof[0]
solution = gep.simplify(hof[0])
print(best_individual)

### The above code cells are implementation of documentation, below cells are the actual code

In [ ]:
!pip install geppy

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 9.1 MB/s eta 0:00:00
  Created wheel for geppy: filename=geppy-0.1.3-py3-none-any.whl size=34965 sha256=088845ef70622e0dccee515188fefd4b8760960f34531b9293e4779c2d52505a
  Stored in directory: /root/.cache/pip/wheels/05/74/8d/ad0e08b902c7faa2fb9011bef1a0efde8c676e7f9b4d2fe67d
Successfully built geppy


In [ ]:
import operator
import numpy as np
import geppy as gep
from deap import creator, base, tools

# Step 1: Define the target function
def target_function(x, y):
    return x**2 + y**2

# Step 2: Define the dataset
x_data = np.linspace(-10, 10, 50)
y_data = np.linspace(-10, 10, 50)
X, Y = np.meshgrid(x_data, y_data)
Z = target_function(X, Y)  # Target outputs

# Flatten the data for evaluation
inputs = np.array([X.ravel(), Y.ravel()]).T
outputs = Z.ravel()

# Step 3: Define the GEP primitive set
pset = gep.PrimitiveSet('main', input_names=['x', 'y'])
#pset.add_function(max, 2)
pset.add_function(operator.add, 2)
pset.add_function(operator.mul, 2)
pset.add_constant_terminal(3)

# Step 4: Define the fitness and individual
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create('Individual', gep.Chromosome, fitness=creator.FitnessMax)

# Step 5: Define the toolbox
toolbox = gep.Toolbox()

# Register chromosome, population, and compile function
toolbox.register('gene_gen', gep.Gene, pset=pset, head_length=h)
toolbox.register('individual', creator.Individual, gene_gen=toolbox.gene_gen, n_genes=n_genes, linker=operator.add)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)
toolbox.register('compile', gep.compile_, pset=pset)

# Define the fitness evaluation function
def evaluate(individual):
    func = toolbox.compile(individual)
    predictions = np.array([func(*input_pair) for input_pair in inputs])
    fitness = -np.mean((outputs - predictions)**2)  # Negative MSE
    return fitness,

toolbox.register('evaluate', evaluate)

# Register selection, mutation, and crossover operators
toolbox.register('select', tools.selRoulette)
toolbox.register('mut_uniform', gep.mutate_uniform, pset=pset, ind_pb=2 / (2 * 10 + 1))
toolbox.pbs['mut_uniform'] = 1
toolbox.register('mut_invert', gep.invert, pb=0.1)
toolbox.register('mut_is_ts', gep.is_transpose, pb=0.1)
toolbox.register('mut_ris_ts', gep.ris_transpose, pb=0.1)
toolbox.register('mut_gene_ts', gep.gene_transpose, pb=0.1)
toolbox.register('cx_1p', gep.crossover_one_point, pb=0.4)
toolbox.register('cx_2p', gep.crossover_two_point, pb=0.2)
toolbox.register('cx_gene', gep.crossover_gene, pb=0.1)

# Step 6: Define statistics and Hall of Fame
stats = tools.Statistics(key=lambda ind: ind.fitness.values[0])
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

hof = tools.HallOfFame(3)
# size of population and number of generations
n_pop = 100
n_gen = 100

pop = toolbox.population(n=n_pop)

# start evolution
pop, log = gep.gep_simple(pop, toolbox, n_generations=n_gen, n_elites=1,
        stats=stats, hall_of_fame=hof, verbose=True)

# Step 8: Output the best individual
best_individual = hof[0]
simplified_solution = gep.simplify(best_individual)

print("\nBest Individual (Chromosome):")
print(best_individual)
print("\nSimplified Solution:")
print(simplified_solution)

# Evaluate the error of the solution
best_func = toolbox.compile(best_individual)
predictions = np.array([best_func(*input_pair) for input_pair in inputs])
mse = np.mean((outputs - predictions)**2)
print(f"\nMean Squared Error of the Best Solution: {mse:.6f}")


gen	nevals	avg         	std        	min         	max     
0  	100   	-6.62559e+06	4.48443e+07	-4.33244e+08	-1020.97
1  	99    	-3.08526e+08	2.58148e+09	-2.58617e+10	-1020.97
2  	99    	-4.0815e+08 	2.82447e+09	-2.81362e+10	-1020.97
3  	99    	-5.72709e+07	2.96972e+08	-1.80375e+09	-1020.97
4  	99    	-1.2742e+07 	1.10373e+08	-1.10452e+09	-1020.97
5  	99    	-1.06651e+08	4.18968e+08	-1.80392e+09	-1020.97
6  	99    	-7.88386e+07	3.44002e+08	-1.82008e+09	-1020.97
7  	99    	-1.07216e+06	5.67509e+06	-5.10646e+07	-1020.97
8  	99    	-4.85864e+07	2.63015e+08	-1.80375e+09	-1020.97
9  	99    	-1.93082e+07	1.79499e+08	-1.80375e+09	-1020.97
10 	99    	-9.5026e+07 	3.79956e+08	-1.80646e+09	-0      
11 	99    	-7.77857e+07	3.54273e+08	-1.80466e+09	-0      
12 	99    	-5.13687e+07	2.86121e+08	-1.80377e+09	-0      
13 	99    	-1.90905e+07	1.79419e+08	-1.80375e+09	-0      
14 	99    	-6.06095e+07	3.09729e+08	-1.80376e+09	-0      
15 	99    	-3.38272e+08	2.81053e+09	-2.81344e+10	-0      
16 	99    	-3.

In [ ]:
rename_labels = {'add': '+', 'sub': '-', 'mul' : '*'}
gep.export_expression_tree(best_individual, rename_labels, file='tree.png')